In [1]:
# started automation onf 3 Sep 2024 19:15
from core.core_log import setup_logging, get_logger
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from core.database import SensorDataHandler
from core.plotting import draw_plots
from core.util import create_timelapse
from core.sensors.camera import RpiCamController, DIMENSION_FULL_HD, DIMENSION_CAMERA_DEFAULT
from core.sensors.dht import get_sensor_data
from core.core_configuration import load_config, database_config, hometemp_config

setup_logging()
load_config()
log = get_logger(__name__)

In [2]:
#live_sensor_data = get_sensor_data(int(hometemp_config()["sensor_pin"]), True)
#live_sensor_data

In [3]:
rpi_cam = RpiCamController()
#name = f'pictures/{datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}'
name = "current"
#rpi_cam.capture_image(file_path=name, rotation=90)
#create_timelapse('./pictures', 'timelapse.mp4')

In [ ]:
auth = database_config()
handler = SensorDataHandler(auth['db_port'], auth['db_host'], auth['db_user'], auth['db_pw'], 'sensor_data')
handler.init_db_connection(check_table=False)
df = handler.read_data_into_dataframe()
df['timestamp'] = df['timestamp'].map(lambda x : datetime.strptime(str(x).replace("+00:00", "").strip(), '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by='timestamp')
df = df.reset_index()

df

In [5]:
# last two hours
#df[["timestamp", "room_temp"]].tail(12)

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [8]:
#draw_plots(df, with_save=False)

In [ ]:
def foo():
    plt.figure(figsize=(25, 10))
    plt.subplot(2, 1, 1)

    sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
    plt.title("Temperature Over Time")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)

    plt.subplot(2, 1, 2)
    sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
    plt.title("Temperature Last 24h")
    plt.xlabel("Time")
    plt.ylabel("Temp (°C)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

#foo()
df

In [ ]:
from core.p import main_plot_params, inner_plots_params, create_multiple_lineplots, _validate_and_sort_timestamp
import pandas as pd
custom_theme = {
    #'context': 'notebook',   # Can be 'paper', 'notebook', 'talk', 'poster'
    'style': 'darkgrid',    # Can be 'darkgrid', 'whitegrid', 'dark', 'white', 'ticks'
    'palette': 'deep'        # Can be 'deep', 'muted', 'bright', 'pastel', etc.
}

# Create single plots with the custom theme
df = _validate_and_sort_timestamp(df)
last_24h = datetime.now() - timedelta(hours=24)
filtered_data = df[df['timestamp'] >= last_24h]
#plot1 = create_single_lineplot(df, "Home", "timestamp", "room_temp", theme=custom_theme)
# Combine multiple plots into one figure with the custom theme
p1 = {"main": main_plot_params(df, "Temperature Over Time")}
p2 = {'main': main_plot_params(filtered_data,"Temperature Last 24 Hours")}
plots_w_params = [p1, p2]
combined_fig, _ = create_multiple_lineplots(plots_w_params, theme=custom_theme, rows=2, cols=1)
combined_fig

#datas = [df, filtered_data]


In [11]:

#create_multiple_plots(datas, max_plots_per_row=2, theme=custom_theme)
# List[Tuple[Figure, Axes]]
def bar(df_list):
    
    rows = 2
    cols = int(len(df_list) / 2) +1 
    print(f"{rows}x{cols}")
    fig, axs = plt.subplots(rows, cols)
    for i, df in enumerate(df_list):   
        row, col = divmod(i, rows)
        print(f"i:{i} row:{row} col:{col}")
        ax_in_subplot = axs[row, col]
        ax_in_subplot.plot(df['timestamp'], df['room_temp'])
        ax_in_subplot.xticks(rotation=45)
        #ax = fig.add_subplot(gs[row, col]) 
        #axes[i].xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M:%S'))
        #sns.lineplot(x=ax_single.lines[0].get_xdata(), y=ax_single.lines[0].get_ydata(), ax=ax)
        #ax.set_xlabel(ax_single.get_xlabel())
        #ax.set_ylabel(ax_single.get_ylabel())
        #ax.set_title(f'Plot {i+1}')
    fig.tight_layout()

#bar(datas)